In [49]:
from collections import defaultdict, OrderedDict
from csv import reader
from itertools import chain, combinations
from optparse import OptionParser
import psutil

# Util functions
## 

In [50]:
from csv import reader
from collections import defaultdict
from itertools import chain, combinations

class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind+1)

## csv reader function

In [51]:
def getFromFile(fname):
    itemSetList = []
    frequency = []
    
    with open(fname, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            itemSetList.append(line)
            frequency.append(1)

    return itemSetList, frequency

## Building the tree

In [52]:
def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    fpTree = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable

def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)

def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1] 
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)  
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next  
    return condPats, frequency

def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])] 
    # Start with the lowest frequency
    for item in sortedItemList:  
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable) 
        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constructTree(conditionalPattBase, frequency, minSup) 
        if newHeaderTable != None:
            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup,
                       newFreqSet, freqItemList)

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    return rules

def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

# Manual FPGrowth

In [53]:
def fpgrowthFromFile(fname, minSupRatio, minConf):
    itemSetList, frequency = getFromFile(fname)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    if fpTree is None:
        print('No frequent item set')
        return [], []
    else:
        freqItems = []
        rules = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf)
        return freqItems, rules
    
freqItemSet, rules = fpgrowthFromFile("Market_Basket_Optimisation.csv", 0.001, 0.4)
if len(freqItemSet) > 0 or len(rules) > 0:
        print("freqItemSet")
        print(freqItemSet)
        print("rules")
        print(rules)

freqItemSet
[{'bramble'}, {'tea'}, {'eggs', 'tea'}, {'frozen vegetables', 'tea'}, {'spaghetti', 'tea'}, {'mineral water', 'tea'}, {'chutney'}, {'spaghetti', 'chutney'}, {'eggs', 'chutney'}, {'chutney', 'mineral water'}, {'mashed potato'}, {'chocolate', 'mashed potato'}, {'mashed potato', 'mineral water'}, {'spaghetti', 'mashed potato'}, {'chocolate bread'}, {'red wine', 'chocolate bread'}, {'mineral water', 'chocolate bread'}, {'dessert wine'}, {'salmon', 'dessert wine'}, {'milk', 'dessert wine'}, {'spaghetti', 'dessert wine'}, {'dessert wine', 'mineral water'}, {'ketchup'}, {'ketchup', 'turkey'}, {'french fries', 'ketchup'}, {'ketchup', 'mineral water'}, {'ketchup', 'milk'}, {'spaghetti', 'ketchup'}, {'ketchup', 'pancakes'}, {'oatmeal'}, {'oatmeal', 'ground beef'}, {'oatmeal', 'green tea'}, {'spaghetti', 'oatmeal'}, {'chocolate', 'oatmeal'}, {'oatmeal', 'mineral water'}, {'sandwich'}, {'spaghetti', 'sandwich'}, {'babies food'}, {'babies food', 'milk'}, {'babies food', 'chocolate'}, {'

## Memory and cpu usage

In [54]:
# Calculate frequent item sets while measuring memory usage
process = psutil.Process()
memory_usage = process.memory_info().rss / 1024 ** 2
print("memory_usage(MB):", memory_usage)

# Calculate the processor usage percentage
cpu_usage = psutil.cpu_percent(interval=1)
print("cpu_percent:", cpu_usage)

memory_usage(MB): 203.28515625
cpu_percent: 1.4


# Built in Algorithm

In [55]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import psutil

In [56]:
df = pd.read_csv("Market_Basket_Optimisation.csv")
df = df.astype(str)
te = TransactionEncoder()
te_ary = te.fit(df.values).transform(df.values)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [57]:
frequent_itemsets = fpgrowth(df, min_support=0.001, use_colnames=True)

In [58]:
print(frequent_itemsets)

        support                                           itemsets
0      1.000000                                              (nan)
1      0.179733                                             (eggs)
2      0.087200                                          (burgers)
3      0.020933                                        (meatballs)
4      0.004133                                          (chutney)
...         ...                                                ...
13508  0.001067  (nan, whole weat flour, mineral water, ground ...
13509  0.001200             (herb & pepper, whole weat flour, nan)
13510  0.001333                  (tomatoes, whole weat flour, nan)
13511  0.001333                      (eggs, whole weat flour, nan)
13512  0.002267                 (spaghetti, whole weat flour, nan)

[13513 rows x 2 columns]


In [59]:
# Calculate frequent item sets while measuring memory usage
process = psutil.Process()
memory_usage = process.memory_info().rss / 1024 ** 2
print("memory_usage(MB):", memory_usage)

# Calculate the processor usage percentage
cpu_usage = psutil.cpu_percent(interval=1)
print("cpu_percent:", cpu_usage)

memory_usage(MB): 207.93359375
cpu_percent: 2.6
